In [54]:
import pandas as pd
import spacy

**Import du fichier généré pour effectuer les tests de NLP et de machine learning**

In [55]:
df = pd.read_csv("../data/nlp_generated_text.csv")

In [56]:
df.head()

,texte,criticité
0,"Je suis blessé, j’ai besoin d’un centre médica...",urgent
1,"Besoin de soins médicaux, c’est urgent. Où all...",urgent
2,Où puis-je rencontrer une maraude ce soir à Pa...,urgent
3,Je cherche les horaires d’une maraude dans Par...,urgent
4,"J’ai des pensées suicidaires, je suis à chemin...",urgent


**Nettoyage du fichier**

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   texte      2000 non-null   object
 1   criticité  2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [58]:
# On vérifie si la colonne contient des doublons et on les traite s'il y en a.

filtered_df = df[df['texte'].isna()]
print(filtered_df)

df = df.dropna()

df.info()



Empty DataFrame
Columns: [texte, criticité]
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   texte      2000 non-null   object
 1   criticité  2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [59]:
# On vérifie qu'on a bien que des valeurs avec un type String

print(df['texte'].apply(type).value_counts()) 

texte
<class 'str'>    2000
Name: count, dtype: int64


**Chargement du modèle français de spaCy**

In [60]:
nlp = spacy.load("fr_core_news_sm")  # Chargement du modèle français


In [61]:
# on applique le modèle spaCy à chaque ligne de la colonne 'texte'

df['spacy_doc'] = df['texte'].apply(lambda x: nlp(x))

df.head()

,texte,criticité,spacy_doc
0,"Je suis blessé, j’ai besoin d’un centre médica...",urgent,"(Je, suis, blessé, ,, j’, ai, besoin, d’, un, ..."
1,"Besoin de soins médicaux, c’est urgent. Où all...",urgent,"(Besoin, de, soins, médicaux, ,, c’, est, urge..."
2,Où puis-je rencontrer une maraude ce soir à Pa...,urgent,"(Où, puis, -je, rencontrer, une, maraude, ce, ..."
3,Je cherche les horaires d’une maraude dans Par...,urgent,"(Je, cherche, les, horaires, d’, une, maraude,..."
4,"J’ai des pensées suicidaires, je suis à chemin...",urgent,"(J’, ai, des, pensées, suicidaires, ,, je, sui..."


In [62]:
for doc in df['spacy_doc']:
    for ent in doc.ents:
        print(ent.label_, ent.text)

MISC j’ai besoin
LOC Besoin
MISC c’
LOC Paris
LOC Paris
LOC Paris
PER Hoarau
PER Élodie Benoit
LOC Paris
MISC quelqu’
PER Agathe Delahaye
LOC Paris
MISC quelqu’
LOC j’
LOC Paris
MISC quelqu’
LOC Dupont
LOC Devaux
LOC Paris
LOC Paris
LOC Paris
PER Thibault Marion
LOC Paris
PER Lebon
PER Breton
LOC Paris
MISC quelqu’
LOC Paris
ORG C’
LOC boulevard de Grenier
LOC Paris
MISC quelqu’
LOC Gomez
LOC Paris
LOC rue Dias
LOC Costa
LOC Paris
MISC quelqu’
LOC Paris
LOC Raymond
MISC Adam
LOC Carpentier
PER Guillaume Texier
LOC Paris
MISC quelqu’
LOC Paris
PER Laurent
PER Arthur Deschamps
LOC Paris
PER Marc Pelletier
LOC Salut
LOC Paris
LOC Paris
LOC Paris
LOC boulevard Anaïs Hervé
PER Jacqueline Loiseau
LOC Paris
MISC quelqu’
LOC Paris
LOC Courtois
MISC Bonjour
LOC qu’
PER Chantal Techer
LOC Paris
PER Tristan Raymond
ORG Delannoy
LOC Masse
LOC Paris
MISC quelqu’
LOC Paris
PER Vallée
LOC rue Mercier
LOC Paris
MISC quelqu’
LOC Paris
PER Christiane Lombard
MISC Bonjour
LOC qu’
LOC boulevard Parent
MIS

In [63]:
df["entites"] = df["spacy_doc"].apply(lambda doc: [(ent.label_, ent.text) for ent in doc.ents])


In [72]:
df.head()

,texte,criticité,spacy_doc,entites,clean_text
0,"Je suis blessé, j’ai besoin d’un centre médica...",urgent,"(Je, suis, blessé, ,, j’, ai, besoin, d’, un, ...","[(MISC, j’ai besoin)]",blesser besoin centre médical rapidement
1,"Besoin de soins médicaux, c’est urgent. Où all...",urgent,"(Besoin, de, soins, médicaux, ,, c’, est, urge...","[(LOC, Besoin), (MISC, c’), (LOC, Paris)]",besoin soin médical urgent aller paris prendre...
2,Où puis-je rencontrer une maraude ce soir à Pa...,urgent,"(Où, puis, -je, rencontrer, une, maraude, ce, ...","[(LOC, Paris)]",je rencontrer maraude soir paris
3,Je cherche les horaires d’une maraude dans Par...,urgent,"(Je, cherche, les, horaires, d’, une, maraude,...","[(LOC, Paris), (PER, Hoarau)]",cherche horaire maraude paris rue hoarau
4,"J’ai des pensées suicidaires, je suis à chemin...",urgent,"(J’, ai, des, pensées, suicidaires, ,, je, sui...","[(PER, Élodie Benoit), (LOC, Paris), (MISC, qu...",pensée suicidaire chemin élodie benoit paris v...


In [65]:
df["clean_text"] = df["spacy_doc"].apply(lambda doc: " ".join([
    token.lemma_.lower() for token in doc
    if not token.is_stop and not token.is_punct
]))

In [66]:
df.head()

,texte,criticité,spacy_doc,entites,clean_text
0,"Je suis blessé, j’ai besoin d’un centre médica...",urgent,"(Je, suis, blessé, ,, j’, ai, besoin, d’, un, ...","[(MISC, j’ai besoin)]",blesser besoin centre médical rapidement
1,"Besoin de soins médicaux, c’est urgent. Où all...",urgent,"(Besoin, de, soins, médicaux, ,, c’, est, urge...","[(LOC, Besoin), (MISC, c’), (LOC, Paris)]",besoin soin médical urgent aller paris prendre...
2,Où puis-je rencontrer une maraude ce soir à Pa...,urgent,"(Où, puis, -je, rencontrer, une, maraude, ce, ...","[(LOC, Paris)]",je rencontrer maraude soir paris
3,Je cherche les horaires d’une maraude dans Par...,urgent,"(Je, cherche, les, horaires, d’, une, maraude,...","[(LOC, Paris), (PER, Hoarau)]",cherche horaire maraude paris rue hoarau
4,"J’ai des pensées suicidaires, je suis à chemin...",urgent,"(J’, ai, des, pensées, suicidaires, ,, je, sui...","[(PER, Élodie Benoit), (LOC, Paris), (MISC, qu...",pensée suicidaire chemin élodie benoit paris v...


In [ ]:
df["Subject"] = df["clean_text"].apply(lambda x: y for )

medical = ["médecin", "douleur", "blessure", "accident"]
medical = ["médecin", "douleur", "blessure", "accident"]

for i in df["clean_text "] :
    if i in medical :
        df["Subject"] = "santé"

# revoir le fichier csv pour avoir une catégorie "sujet" (médical, commissariat, maraude, douche")
# Utiliser un algo de classification pour catégoriser chaque ligne


In [67]:
# Pas sûre de garder cette partie sur TfidfVectorizer. On a trop de lignes et trop de caractères par ligne pour pouvoir voir quelque chose j pense

#from sklearn.feature_extraction.text import TfidfVectorizer

#vectorizer = TfidfVectorizer()
#X = vectorizer.fit_transform(df["clean_text"])

In [68]:
#print(vectorizer.get_feature_names_out())  # Liste des mots

In [69]:
#print(X.toarray()[10:30])  # Matrice TF-IDF sur un échantillon

In [70]:
""""
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
 
result = classifier("J'adore ce produit, il est excellent !")
print(result)

"""


'"\nfrom transformers import pipeline\nclassifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")\n\nresult = classifier("J\'adore ce produit, il est excellent !")\nprint(result)\n\n'